In [1]:
import torchvision.models as models
import torch
import torch.nn as nn

from alexnet import load_alexnet
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision

import numpy as np
import matplotlib.pyplot as plt

from hasher import Hasher,set_conv_weights,get_conv_weights
from utils import cifar10_loader,evaluate

In [2]:
model = load_alexnet().cuda()
train_loader,test_loader = cifar10_loader()

In [ ]:
train_acc = evaluate(model,train_loader,True)
print(f"\ttrain Accuracy : {train_acc}")


In [3]:
weights,biases,ip_sizes,names = get_conv_weights(model)

In [4]:
hasher = Hasher(ip_sizes,16)
hasher.cuda()
hasher.init()

In [5]:
hasher.eval()

Hasher(
  (hashnet): HashNet(
    (encoder): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
      (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
    (op): Sequential(
      (0): Linear(in_features=1024, out_features=256, bias=True)
      (1): Linear(in_features=256, out_features=64, bias=True)
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): Linear(in_features=64, out_features=16, bias=True)
    )
  )
  (centroid_net): CentroidNet(
    

In [9]:
new_weights = []
for weight in weights:
    new_weight,x_up,Q,prob,bins = hasher(weight)
    new_weights.append(new_weight)

In [10]:
model = set_conv_weights(model,new_weights,biases,names=names)

In [11]:
train_acc = evaluate(model,train_loader,True)
print(f"\ttrain Accuracy : {train_acc}")


	train Accuracy : 0.1
